# Molecular Morphing and Fingerprinting

This jupyternotebook includes molecular morphing from G0 molecule PDMA to generate hypothetical molecules in up to G6.

In [1]:
import sys
sys.path.append('../03-code/')
from config import PROJECT_ROOT_DIRECTORY, COLUMNS_DICT

import csv
import pandas as pd

#### Obtain SMILES of organic spacers generated from PDMA

In [ ]:
from rdkit import Chem
from rdkit.Chem import AllChem

# define the reaction smarts dictionary from PDMA as G0

reaction_smarts_dict = {
    1: AllChem.ReactionFromSmarts('[NH3+:1][C:2]c1ccc([C:3][NH3+:4])cc1>>[NH3+:1][C:2]c1sc([C:3][NH3+:4])cc1'), # benzene thiophene ring exchange 1
    
    2: AllChem.ReactionFromSmarts('[NH3+:1][C:2]c1ccc([C:3][NH3+:4])cc1>>[NH3+:1][C:2]c1scc([C:3][NH3+:4])c1'), # benzene thiophene ring exchange 2

    3: AllChem.ReactionFromSmarts('[c:1][C:2][NH3+:3]>>[c:1](-[c]1[ch]cc([C:2][NH3+:3])c[ch]1)'), # benzene ring linkage
    4: AllChem.ReactionFromSmarts('[c:1][C:2][NH3+:3]>>[c:1]c1csc([C:2][NH3+:3])c1'), # thiophen ring linkage 1
    5: AllChem.ReactionFromSmarts('[c:1][C:2][NH3+:3]>>[c:1]c1sc([C:2][NH3+:3])cc1'), # thiophene ring linkage 2
    6: AllChem.ReactionFromSmarts('[cH:1][c:2][C:3][NH3+:4]>>[c:1]2ccc([C:3][NH3+:4])c[c:2]2'),  # benzene ring fusion
    7: AllChem.ReactionFromSmarts('[cH:1][c:2][C:3][NH3+:4]>>[c:1]2sc([C:3][NH3+:4])c[c:2]2'), # thiophene ring fusion

    8: AllChem.ReactionFromSmarts('[cH:1][c:2][C:3][NH3+:4]>>[cH:2][c:1][C:3][NH3+:4]'), # linker position change

    9: AllChem.ReactionFromSmarts('[c:1][C][NH3+]>>[nH+:1]'), # tethering ammonium exchange

    10: AllChem.ReactionFromSmarts('[#6:1][C:2][NH3+:3]>>[#6:1][C:2][CH2][NH3+:3]'), # linker length increase
    11: AllChem.ReactionFromSmarts('[#6:1][C][NH3+:3]>>[#6:1][NH3+:3]'), # linker length decrease

    12: AllChem.ReactionFromSmarts('[cH:1]>>[n:1]'), # nitrogen substitution

    13: AllChem.ReactionFromSmarts('[s&r5:1]>>[o&r5:1]'), # furan thiophene ring exchange

    14: AllChem.ReactionFromSmarts('[s&r5:1]>>[nH:1]'), # pyrrole thiophene ring exchange

    15: AllChem.ReactionFromSmarts('[cH:1]>>[c:1][F]'), # fluorination

    16: AllChem.ReactionFromSmarts('[#6,#7:1][CH2:2][#6,#7:3]>>[#6,#7:1][CH:2](C)[#6,#7:3]'), # side chain addition on linker
    
    17: AllChem.ReactionFromSmarts('[cH:1]>>[c:1]C'), # side chain addition on backbone

}

In [ ]:
# starting from PDMA 
from collections import Counter

def is_valid_modification(sequence, next_mod):
    """
    Validate if the next modification is allowed based on the sequence and new constraints.
    
    Optimized Constraints:
    - count(1) + count(2) <= 1
    - count(9) <= 1
    - count(13 + 14) <= count(1 + 2 + 4 + 5 + 7)
    - next modification >= last modification except for when last modification is in range 3-7, next modification >= 3.
    - The counts should include both previous modifications and the next modification.
    """
    # Use Counter to get counts of all elements in the sequence efficiently
    counts = Counter(sequence)
    
    # Increment the count of the next modification
    counts[next_mod] += 1

    # Constraint 1: count(1) + count(2) <= 1
    if counts[1] + counts[2] > 1:
        return False

    # Constraint 2: count(9) <= 1
    if counts[9] > 1:
        return False

    # Constraint 3: count(13 + 14) <= count(1 + 2 + 4 + 5 + 7)
    if counts[13] + counts[14] > counts[1] + counts[2] + counts[4] + counts[5] + counts[7]:
        return False

    # Get the last modification in the sequence, or 0 if the sequence is empty
    last_mod = sequence[-1] if sequence else 0

    # Constraint 5: Next modification must be >= last modification
    # except when the last modification is in the range 3-7, next modification must be >= 3.
    if last_mod >= 3 and last_mod <= 7:
        if next_mod < 3:
            return False
    elif next_mod < last_mod:
        return False

    # All constraints satisfied
    return True

def apply_all_reactions_once_with_sequence(input_smiles, sequence):
    # input one smiles string and its sequence
    mol = Chem.MolFromSmiles(input_smiles)
    
    unique_smiles_set = set()  # To store unique (SMILES, sequence) pairs
    valid_smiles = [] 

    for next_mod in range(1, 18):
        if is_valid_modification(sequence, next_mod):
            rxn = reaction_smarts_dict[next_mod]
            products = rxn.RunReactants((mol,))

            for product_set in products:
                try:
                    Chem.SanitizeMol(product_set[0])
                    smiles = Chem.MolToSmiles(product_set[0])
                    smiles_sequence_pair = (smiles, tuple(sequence + [next_mod]))

                    # Ensure that only unique SMILES and sequences are kept
                    if smiles_sequence_pair not in unique_smiles_set:
                        unique_smiles_set.add(smiles_sequence_pair)
                        valid_smiles.append((smiles, sequence + [next_mod]))
                except:
                    pass
    
    return valid_smiles

def iterative_morphing_operations_to_csv(starting_smiles, sequence, num_iterations, output_file_prefix):
    current_smiles_list = [(starting_smiles, sequence)]
    all_seen_smiles = set([starting_smiles])

    for iteration in range(1, num_iterations + 1):
        new_smiles_list = []

        for smiles, seq in current_smiles_list:
            valid_smiles = apply_all_reactions_once_with_sequence(smiles, seq)

            for new_smiles, new_seq in valid_smiles:
                if new_smiles not in all_seen_smiles:
                    new_smiles_list.append((new_smiles, new_seq))
                    all_seen_smiles.add(new_smiles)

        output_file = f"{output_file_prefix}_generation_{iteration}.csv"
        with open(output_file, 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(['smiles_canonical', 'Sequence'])
            for smiles, seq in new_smiles_list:
                writer.writerow([smiles, seq])

        print(f"Generation {iteration} completed. Results saved to {output_file}")
        current_smiles_list = new_smiles_list
    return list(all_seen_smiles)

In [ ]:
starting_mol = Chem.MolFromSmiles('[NH3+]Cc1ccc(C[NH3+])cc1') # this is the SMIELS of PDMA
starting_smiles = Chem.MolToSmiles(starting_mol)
starting_sequence = []

smiles_list = iterative_morphing_operations_to_csv(starting_smiles, starting_sequence, num_iterations=7, output_file_prefix=PROJECT_ROOT_DIRECTORY+'01-rawdata/01-molecular-generation/smiles')

#### Visualize the generated SMILES

In [2]:
df = pd.read_csv(PROJECT_ROOT_DIRECTORY+'01-rawdata/01-molecular-generation/smiles_generation_'+str(4)+'.csv')

In [3]:
# figure out the number of smiles generated in each iteration
smiles_count = {}
for i in range(0, 8):
    df = pd.read_csv(PROJECT_ROOT_DIRECTORY+'01-rawdata/01-molecular-generation/smiles_generation_'+str(i)+'.csv')
    smiles_count[i] = df.shape[0]
smiles_count

{0: 1, 1: 15, 2: 236, 3: 2987, 4: 35495, 5: 401932, 6: 4446434, 7: 48414134}

In [ ]:
from utils import visualize_smiles_list

i=2
input_csv = PROJECT_ROOT_DIRECTORY + '01-rawdata/01-molecular-generation/smiles_generation_' + str(i) + '.csv'
# visualize the structure of organic molecules
df = pd.read_csv(input_csv)
smiles_list = df['smiles_canonical'].tolist()
visualize_smiles_list(smiles_list[:50])

#### Obtain the fingerprint of smiles string

In [2]:
from organic_featurization_helper import get_organic_structure_descriptors

def generate_fingerprint_csv_file_from_smiles(input_csv, output_csv):
    """
    Read the SMILES and sequences from an input CSV, compute the fingerprint vectors (organic descriptors),
    and write the results to an output CSV file.
    
    Parameters:
    - input_csv: Path to the input CSV containing SMILES and sequences.
    - output_csv: Path to the output CSV file where results will be saved.
    """
    with open(input_csv, 'r') as infile, open(output_csv, 'w', newline='') as outfile:
        reader = csv.DictReader(infile)
        fieldnames = ['smiles_canonical'] + COLUMNS_DICT['molecular_fingerprint']# + ['HOMO_prediction','LUMO_prediction']  # Add functional group names as columns
        writer = csv.DictWriter(outfile, fieldnames=fieldnames)
        
        # Write the header to the output CSV
        writer.writeheader()
        
        # Process each row (SMILES and sequence) in the input CSV
        for row in reader:
            smiles = row['smiles_canonical']
            
            # Calculate the fingerprint vector (counts of functional groups)
            organic_structure_descriptors = get_organic_structure_descriptors(smiles)
                            # Prepare the output row containing SMILES, sequence, and fingerprint vector
            fingerprint_dict = {key: organic_structure_descriptors[key] for key in COLUMNS_DICT['molecular_fingerprint']}
            output_row = {
                'smiles_canonical': smiles,
            }
            
            output_row.update(fingerprint_dict)  # Add fingerprint data to the row
                
            # Write the row to the output CSV
            writer.writerow(output_row)


In [3]:
# obtain fingerprint to each iteration and save the results
for i in range(5,7):
    input_csv = PROJECT_ROOT_DIRECTORY + '01-rawdata/01-molecular-generation/smiles_generation_' + str(i) + '.csv'  # Input file containing SMILES and sequences
    output_csv = PROJECT_ROOT_DIRECTORY + '01-rawdata/01-molecular-generation/fingerprints/fingerprints_generation_' + str(i) + '.csv' # Output file to store SMILES and fingerprint vectors

    # Process the input CSV and output the fingerprint vectors to a new CSV
    generate_fingerprint_csv_file_from_smiles(input_csv, output_csv)


### log information into organic genome dataframe

In [2]:
organic_genome_dataframe = pd.read_csv(
    PROJECT_ROOT_DIRECTORY + '02-metadata/06-csv-files/01-organic-genome.csv', index_col='identifier'
)

In [74]:
identifier_list = organic_genome_dataframe.query('generation == 2').index.to_list()
print(identifier_list)
# change the format to string and then remove the blank space in this string
str(identifier_list).replace(' ','').replace('[','').replace(']','')

[1, 5, 6, 8, 9, 10, 11, 17, 18, 21, 24, 27, 29, 31, 33, 35, 36, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 56, 57, 62, 63, 64, 68, 85, 89, 90, 97, 100, 103, 106, 108, 112, 113, 114, 115, 116, 118, 120, 122, 131, 133, 141, 161, 168, 175, 183, 191, 193, 194, 195, 196, 203, 204, 212, 216, 217, 218, 221, 222, 223, 224, 228, 229, 234, 258, 260, 262, 267, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362, 363, 364, 365, 366, 367, 368, 369, 370, 371, 372, 373, 374, 375, 376, 377, 378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 389, 390, 391, 392, 393, 394, 395, 396, 397, 

'1,5,6,8,9,10,11,17,18,21,24,27,29,31,33,35,36,39,40,41,42,43,44,45,46,47,48,49,50,51,52,56,57,62,63,64,68,85,89,90,97,100,103,106,108,112,113,114,115,116,118,120,122,131,133,141,161,168,175,183,191,193,194,195,196,203,204,212,216,217,218,221,222,223,224,228,229,234,258,260,262,267,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,1822,1823,1824,1825,1826,1827,1828,1829,1830,1831,1832,1833,1834,1835,1836,1837,1838,1839,1840,1841,1842,1843,1844'

In [7]:
organic_genome_dataframe

,smiles_canonical,generation
identifier,,
1,[NH3+]Cc1cc2sc(C[NH3+])cc2s1,2.0
2,[NH3+]Cc1cc2sc([NH3+])cc2s1,3.0
3,[NH3+]c1cc2sc([NH3+])cc2s1,4.0
4,[NH3+]Cc1ccc(C[NH3+])s1,1.0
5,[NH3+]Cc1ccc(C[NH3+])o1,2.0
...,...,...
40605,[NH3+]Cc1ccc2nc(-c3cs[nH+]n3)[nH]c2c1,6.0
40606,[NH3+]Cc1ccc2nc(-c3cn[nH+]s3)[nH]c2c1,6.0
40607,Cc1c[nH+]nc2oc(C[NH3+])nc12,6.0


In [37]:
from rdkit import Chem
new_mol = Chem.MolFromSmiles('[NH3+]CC1=C(F)C(F)=C2C(C(F)=C(C(F)=C(C(F)=C(C(F)=C(F)C(C[NH3+])=C3F)C3=C4F)C4=C5F)C5=C2F)=C1F') 
new_smiles = Chem.MolToSmiles(new_mol)
new_smiles

'[NH3+]Cc1c(F)c(F)c2c(F)c3c(F)c4c(F)c5c(F)c(C[NH3+])c(F)c(F)c5c(F)c4c(F)c3c(F)c2c1F'

In [38]:
# add new row to organic genome dataframe
new_row = {
    'smiles_canonical': new_smiles,
    'generation': 16,
}
organic_genome_dataframe.loc[40620] = new_row
organic_genome_dataframe

,smiles_canonical,generation
identifier,,
1,[NH3+]Cc1cc2sc(C[NH3+])cc2s1,2.0
2,[NH3+]Cc1cc2sc([NH3+])cc2s1,3.0
3,[NH3+]c1cc2sc([NH3+])cc2s1,4.0
4,[NH3+]Cc1ccc(C[NH3+])s1,1.0
5,[NH3+]Cc1ccc(C[NH3+])o1,2.0
...,...,...
40616,[NH3+]Cc1sc(-c2sc(-c3sc(-c4sc(-c5sc(C[NH3+])c(...,15.0
40617,[NH3+]Cc1c(F)c(F)c2c(F)c(C[NH3+])c(F)c(F)c2c1F,7.0
40618,[NH3+]Cc1c(F)c(F)c2c(F)c3c(F)c(C[NH3+])c(F)c(F...,10.0


In [33]:
organic_genome_dataframe.query("smiles_canonical == @new_smiles")

,smiles_canonical,generation
identifier,,


In [3]:
dataframe_combined = pd.DataFrame()
for i in range(0, 5):
    df_gen = pd.read_csv(PROJECT_ROOT_DIRECTORY+'01-rawdata/01-molecular-generation/smiles_generation_'+str(i)+'.csv')
    df_gen['generation'] = i
    dataframe_combined = pd.concat([dataframe_combined, df_gen], ignore_index=True)

dataframe_combined
    

,SMILES,Sequence,generation
0,[NH3+]Cc1ccc(C[NH3+])cc1,[],0
1,[NH3+]Cc1ccc(C[NH3+])s1,[1],1
2,[NH3+]Cc1csc(C[NH3+])c1,[2],1
3,[NH3+]Cc1ccc(-c2ccc(C[NH3+])cc2)cc1,[3],1
4,[NH3+]Cc1ccc(-c2csc(C[NH3+])c2)cc1,[4],1
...,...,...,...
38729,Cc1cc(C(C)[NH3+])c(C)cc1C(C)[NH3+],"[16, 16, 17, 17]",4
38730,Cc1cc(C(C)[NH3+])cc(C)c1C(C)[NH3+],"[16, 16, 17, 17]",4
38731,Cc1cc(C(C)[NH3+])c(C)c(C)c1C[NH3+],"[16, 17, 17, 17]",4
38732,Cc1cc(C[NH3+])c(C)c(C)c1C(C)[NH3+],"[16, 17, 17, 17]",4


In [23]:
dataframe_combined[~dataframe_combined['smiles_canonical'].isin(organic_genome_dataframe['smiles_canonical'])]

,SMILES,Sequence,generation


In [61]:
# according to the 'generation' column in dataframe_combined, update the 'generation' column in organic_genome_dataframe
for smiles_canonical in dataframe_combined['smiles_canonical']:
    #print(smiles_canonical)
    if smiles_canonical in organic_genome_dataframe.smiles_canonical.to_list():
        generation = dataframe_combined[dataframe_combined['smiles_canonical']==smiles_canonical]['generation'].values[0]
        #print(generation)
        organic_genome_dataframe.loc[organic_genome_dataframe['smiles_canonical']==smiles_canonical, 'generation'] = generation


In [39]:
organic_genome_dataframe

,smiles_canonical,generation
identifier,,
1,[NH3+]Cc1cc2sc(C[NH3+])cc2s1,2.0
2,[NH3+]Cc1cc2sc([NH3+])cc2s1,3.0
3,[NH3+]c1cc2sc([NH3+])cc2s1,4.0
4,[NH3+]Cc1ccc(C[NH3+])s1,1.0
5,[NH3+]Cc1ccc(C[NH3+])o1,2.0
...,...,...
40616,[NH3+]Cc1sc(-c2sc(-c3sc(-c4sc(-c5sc(C[NH3+])c(...,15.0
40617,[NH3+]Cc1c(F)c(F)c2c(F)c(C[NH3+])c(F)c(F)c2c1F,7.0
40618,[NH3+]Cc1c(F)c(F)c2c(F)c3c(F)c(C[NH3+])c(F)c(F...,10.0


In [40]:
organic_genome_dataframe.to_csv(PROJECT_ROOT_DIRECTORY + '02-metadata/06-csv-files/01-organic-genome.csv')

### Different choice of G0 spacers

In [2]:
G0_choice_dataframe = pd.read_csv(
    PROJECT_ROOT_DIRECTORY + '01-rawdata/14-molecular-generation/choice-of-generation.csv', index_col=0
)
G0_choice_dataframe

,gen0,gen1,gen2,gen3,gen4,gen5,gen6,gen7,gen8,gen9,gen10,gen11
index,,,,,,,,,,,,
1,1,1,3,4,7,1,2,1,1,0,0,0
2,1,1,1,7,3,3,2,2,1,0,0,0
3,1,3,8,3,2,3,1,0,0,0,0,0
4,1,0,5,4,7,2,2,0,0,0,0,0
5,1,0,5,4,8,1,1,1,0,0,0,0
6,1,0,0,0,3,3,6,3,1,3,1,0
7,1,2,4,7,2,2,2,1,0,0,0,0
8,1,3,1,4,7,0,3,1,1,0,0,0
9,1,1,3,2,4,5,3,0,2,0,0,0


In [3]:
import plotly.express as px

# Prepare the data
melted_df = G0_choice_dataframe.melt(var_name='Column', value_name='Value', ignore_index=False).reset_index()

fig = px.area(
    melted_df,
    x='Column',
    y='Value',
    color='index',
    facet_col='index',  # Creates separate plots per identifier
    facet_col_wrap=3,        # Wrap columns for better layout
    markers=True,
)

fig.update_layout(template='simple_white',height=800, width=1200)

fig.show()
fig.write_image('../rawfigures/choice-of-gen0.svg')